In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

#geopandas to process geospatial/gis data
import geopandas as gpd

In [2]:
#User input
student_acc_by_age_csv = r"N:\Geodatabase\Raw_Data\Census 2021\Students\LC4411EW - Student accommodation by age - Census 2011.csv"
student_pop_2021_csv = r"N:\Geodatabase\Raw_Data\Census 2021\Students\lsoa2021_2019_lookup_table_with_student_total.csv"

In [9]:
student_by_age_df = pd.read_csv(student_acc_by_age_csv, skiprows = 8, nrows = 34753, skip_blank_lines = True)
student_by_age_4_15_df = pd.read_csv(student_acc_by_age_csv, skiprows = 34784, nrows = 34753, skip_blank_lines = True)
student_by_age_16_17_df = pd.read_csv(student_acc_by_age_csv, skiprows = 69560, nrows = 34753, skip_blank_lines = True)
student_by_age_18_20_df = pd.read_csv(student_acc_by_age_csv, skiprows = 104336, nrows = 34753, skip_blank_lines = True)
student_by_age_20_24_df = pd.read_csv(student_acc_by_age_csv, skiprows = 139112, nrows = 34753, skip_blank_lines = True)
student_by_age_25_above_df = pd.read_csv(student_acc_by_age_csv, skiprows = 173888, nrows = 34753, skip_blank_lines = True)

student_by_age_25_above_df.tail()

,2011 super output area - lower layer,All categories: Student accommodation,Living with parents,Living in a communal establishment: Total,Living in a communal establishment: University (for example halls of residence),Living in a communal establishment: Other,Living in all student household,Student living alone,Living in other household type
34748,W01001954 : Cardiff 006F,13,7,0,0,0,1,0,5
34749,W01001955 : Swansea 025F,90,1,1,0,1,29,48,11
34750,W01001956 : Swansea 023E,58,1,41,41,0,4,1,11
34751,W01001957 : Swansea 025G,55,1,2,0,2,19,15,18
34752,W01001958 : Swansea 025H,81,0,0,0,0,39,28,14


In [10]:
df_list = [
    student_by_age_df,
    student_by_age_4_15_df,
    student_by_age_16_17_df,
    student_by_age_18_20_df,
    student_by_age_20_24_df,
    student_by_age_25_above_df
]


In [11]:
def add_percentage_columns(df, total_col, suffix='_count', new_suffix='_perc', columns_to_convert=None):
    
    if columns_to_convert is None:
        columns_to_convert = [col for col in df.columns if col.endswith(suffix) and col != total_col]

    for col in columns_to_convert:
        perc_col = col.replace(suffix, new_suffix)
        df[perc_col] = (df[col] / df[total_col]) * 100

    return df

In [12]:
# Dictionary for renaming columns with corrected keys
column_rename_map1 = {
    "all_categories:_student_accommodation_count": "total_students_count",
    "living_with_parents_count": "living_with_parents_count",
    "living_in_a_communal_establishment:_university_(for_example_halls_of_residence)_count": "living_in_communal_establishment_uni_count",
    "living_in_a_communal_establishment:_other_count": "living_in_communal_establishment_other_count",    
    "living_in_all_student_household_count": "living_in_all_student_household_count",
    "student_living_alone_count": "living_alone_count",
    "living_in_other_household_type_count": "living_in_other_household_count",
}

column_rename_map2 = {
    "all_categories:_student_accommodation_count": "total_students_age_4_15_count",
    "living_with_parents_count": "living_with_parents_age_4_15_count",
    "living_in_a_communal_establishment:_university_(for_example_halls_of_residence)_count": "living_in_communal_establishment_uni_age_4_15_count",
    "living_in_a_communal_establishment:_other_count": "living_in_communal_establishment_other_age_4_15_count",    
    "living_in_all_student_household_count": "living_in_all_student_household_age_4_15_count",
    "student_living_alone_count": "living_alone_age_4_15_count",
    "living_in_other_household_type_count": "living_in_other_household_age_4_15_count",
}

column_rename_map3 = {
    "all_categories:_student_accommodation_count": "total_students_age_16_17_count",
    "living_with_parents_count": "living_with_parents_age_16_17_count",
    "living_in_a_communal_establishment:_university_(for_example_halls_of_residence)_count": "living_in_communal_establishment_uni_age_16_17_count",
    "living_in_a_communal_establishment:_other_count": "living_in_communal_establishment_other_age_16_17_count",    
    "living_in_all_student_household_count": "living_in_all_student_household_age_16_17_count",
    "student_living_alone_count": "living_alone_age_16_17_count",
    "living_in_other_household_type_count": "living_in_other_household_age_16_17_count",
}

column_rename_map4 = {
    "all_categories:_student_accommodation_count": "total_students_age_18_19_count",
    "living_with_parents_count": "living_with_parents_age_18_19_count",
    "living_in_a_communal_establishment:_university_(for_example_halls_of_residence)_count": "living_in_communal_establishment_uni_age_18_19_count",
    "living_in_a_communal_establishment:_other_count": "living_in_communal_establishment_other_age_18_19_count",  
    "living_in_all_student_household_count": "living_in_all_student_household_age_18_19_count",
    "student_living_alone_count": "living_alone_age_18_19_count",
    "living_in_other_household_type_count": "living_in_other_household_age_18_19_count",
}

column_rename_map5 = {
    "all_categories:_student_accommodation_count": "total_students_age_20_24_count",
    "living_with_parents_count": "living_with_parents_age_20_24_count",
    "living_in_a_communal_establishment:_university_(for_example_halls_of_residence)_count": "living_in_communal_establishment_uni_age_20_24_count",
    "living_in_a_communal_establishment:_other_count": "living_in_communal_establishment_other_age_20_24_count", 
    "living_in_all_student_household_count": "living_in_all_student_household_age_20_24_count",
    "student_living_alone_count": "living_alone_age_20_24_count",
    "living_in_other_household_type_count": "living_in_other_household_age_20_24_count",
}

column_rename_map6 = {
    "all_categories:_student_accommodation_count": "total_students_age_25_above_count",
    "living_with_parents_count": "living_with_parents_age_25_above_count",
    "living_in_a_communal_establishment:_university_(for_example_halls_of_residence)_count": "living_in_communal_establishment_uni_age_25_above_count",
    "living_in_a_communal_establishment:_other_count": "living_in_communal_establishment_other_age_25_above_count",
    "living_in_all_student_household_count": "living_in_all_student_household_age_25_above_count",
    "student_living_alone_count": "living_alone_age_25_above_count",
    "living_in_other_household_type_count": "living_in_other_household_age_25_above_count",
}


rename_list = [column_rename_map1,
               column_rename_map2,
               column_rename_map3,
               column_rename_map4,
               column_rename_map5,
               column_rename_map6,
]

totals_list = ["total_students_count",
               "total_students_age_4_15_count",
               "total_students_age_16_17_count",
               "total_students_age_18_19_count",
               "total_students_age_20_24_count",
               "total_students_age_25_above_count",
]             
               
               
            

In [13]:
for i, df in enumerate(df_list):
    # Split the lsoa column into two new columns
    df[['lsoa11cd', 'lsoa11nm']] = df.iloc[:, 0].str.split(' : ', expand=True)
    
    # Remove the column '2021 super output area - lower layer'
    df.drop(['2011 super output area - lower layer'], 1,  inplace=True)
    
    # Rename columns 
    cols_to_rename = df.columns.difference(['lsoa11cd', 'lsoa11nm'])
    df.rename(columns={col: col.lower().replace(' ', '_') + '_count' for col in cols_to_rename}, inplace=True)
    df.rename(columns={col: col.lower().replace(':', '') for col in cols_to_rename}, inplace=True)
    
    # Remove the column '2021 super output area - lower layer'
    df.drop(['lsoa11nm','living_in_a_communal_establishment:_total_count'], 1,  inplace=True)
    
    # Move 'lsoa21cd' and 'lsoa21nm' to the front of the dataframe
    cols = df.columns.tolist()
    new_order = [cols[-1]] + cols[:-1]
    df_list[i] = df[new_order]   
    
    # Rename columns using the dictionary
    df_list[i].rename(columns=rename_list[i], inplace=True)

    # Add percentage column
    add_percentage_columns(df_list[i], totals_list[i], suffix='_count', new_suffix='_perc', columns_to_convert=df_list[i].columns[-7:])



C:\Users\abhimanya.achara\AppData\Local\Temp\ipykernel_30348\971976749.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(['2011 super output area - lower layer'], 1,  inplace=True)
C:\Users\abhimanya.achara\AppData\Local\Temp\ipykernel_30348\971976749.py:14: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(['lsoa11nm','living_in_a_communal_establishment:_total_count'], 1,  inplace=True)
C:\Users\abhimanya.achara\AppData\Local\Temp\ipykernel_30348\971976749.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(['2011 super output area - lower layer'], 1,  inplace=True)
C:\Users\abhimanya.achara\AppData\Local\Temp\ipykernel_30348\971976749.py:14: FutureWarning: In a future version of pandas all arguments of 

In [14]:
from functools import reduce

# Merge all DataFrames in df_list on 'lsoa21cd', keeping the order from the first df
merged_df = reduce(
    lambda left, right: pd.merge(left, right, on='lsoa11cd', how='left'),
    df_list
)

cols_to_drop = [
               "total_students_perc",
               "total_students_age_4_15_perc",
               "total_students_age_16_17_perc",
               "total_students_age_18_19_perc",
               "total_students_age_20_24_perc",
               "total_students_age_25_above_perc",
]   

merged_df.drop(cols_to_drop,1,inplace = True)

merged_df.head()

C:\Users\abhimanya.achara\AppData\Local\Temp\ipykernel_30348\272398875.py:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  merged_df.drop(cols_to_drop,1,inplace = True)


,lsoa11cd,total_students_count,living_with_parents_count,living_in_communal_establishment_uni_count,living_in_communal_establishment_other_count,living_in_all_student_household_count,living_alone_count,living_in_other_household_count,living_with_parents_perc,living_in_communal_establishment_uni_perc,...,living_in_communal_establishment_other_age_25_above_count,living_in_all_student_household_age_25_above_count,living_alone_age_25_above_count,living_in_other_household_age_25_above_count,living_with_parents_age_25_above_perc,living_in_communal_establishment_uni_age_25_above_perc,living_in_communal_establishment_other_age_25_above_perc,living_in_all_student_household_age_25_above_perc,living_alone_age_25_above_perc,living_in_other_household_age_25_above_perc
0,E01000001,130,89,0,0,10,13,18,68.461538,0.0,...,0,0,6,9,11.764706,0.0,0.000000,0.000000,35.294118,52.941176
1,E01000002,146,111,0,0,4,10,21,76.027397,0.0,...,0,1,5,14,0.000000,0.0,0.000000,5.000000,25.000000,70.000000
2,E01000003,188,78,0,65,15,16,14,41.489362,0.0,...,4,6,6,12,6.666667,0.0,13.333333,20.000000,20.000000,40.000000
3,E01000005,211,166,0,0,18,8,19,78.672986,0.0,...,0,5,3,9,26.086957,0.0,0.000000,21.739130,13.043478,39.130435
4,E01000006,421,328,0,0,20,5,68,77.909739,0.0,...,0,11,3,44,10.769231,0.0,0.000000,16.923077,4.615385,67.692308


In [16]:
student_pop_2021_df = pd.read_csv(student_pop_2021_csv)
student_pop_2021_df.head().head()

,lsoa21cd,lsoa21nm,lsoa11cd,lsoa11nm,total_students_2021_count
0,E01000001,City of London 001A,E01000001,City of London 001A,154
1,E01000002,City of London 001B,E01000002,City of London 001B,141
2,E01000003,City of London 001C,E01000003,City of London 001C,137
3,E01000005,City of London 001E,E01000005,City of London 001E,250
4,E01000006,Barking and Dagenham 016A,E01000006,Barking and Dagenham 016A,416


In [17]:
student_pop_2021_df.shape

(35672, 5)

In [19]:
student_pop_11_df = student_pop_2021_df.groupby('lsoa11cd').sum(numeric_only=True).reset_index()
student_pop_11_df.head()

,lsoa11cd,total_students_2021_count
0,E01000001,154
1,E01000002,141
2,E01000003,137
3,E01000005,250
4,E01000006,416


In [20]:
student_pop_11_df.shape

(34633, 2)

In [22]:
combined_df = merged_df.merge(student_pop_11_df, on = 'lsoa11cd')
combined_df.head()

,lsoa11cd,total_students_count,living_with_parents_count,living_in_communal_establishment_uni_count,living_in_communal_establishment_other_count,living_in_all_student_household_count,living_alone_count,living_in_other_household_count,living_with_parents_perc,living_in_communal_establishment_uni_perc,...,living_in_all_student_household_age_25_above_count,living_alone_age_25_above_count,living_in_other_household_age_25_above_count,living_with_parents_age_25_above_perc,living_in_communal_establishment_uni_age_25_above_perc,living_in_communal_establishment_other_age_25_above_perc,living_in_all_student_household_age_25_above_perc,living_alone_age_25_above_perc,living_in_other_household_age_25_above_perc,total_students_2021_count
0,E01000001,130,89,0,0,10,13,18,68.461538,0.0,...,0,6,9,11.764706,0.0,0.000000,0.000000,35.294118,52.941176,154
1,E01000002,146,111,0,0,4,10,21,76.027397,0.0,...,1,5,14,0.000000,0.0,0.000000,5.000000,25.000000,70.000000,141
2,E01000003,188,78,0,65,15,16,14,41.489362,0.0,...,6,6,12,6.666667,0.0,13.333333,20.000000,20.000000,40.000000,137
3,E01000005,211,166,0,0,18,8,19,78.672986,0.0,...,5,3,9,26.086957,0.0,0.000000,21.739130,13.043478,39.130435,250
4,E01000006,421,328,0,0,20,5,68,77.909739,0.0,...,11,3,44,10.769231,0.0,0.000000,16.923077,4.615385,67.692308,416


In [23]:
combined_df['change_in_student_population_10_yr'] = ((combined_df['total_students_2021_count'] - combined_df['total_students_count'])/combined_df['total_students_count'])*100
combined_df.head()

,lsoa11cd,total_students_count,living_with_parents_count,living_in_communal_establishment_uni_count,living_in_communal_establishment_other_count,living_in_all_student_household_count,living_alone_count,living_in_other_household_count,living_with_parents_perc,living_in_communal_establishment_uni_perc,...,living_alone_age_25_above_count,living_in_other_household_age_25_above_count,living_with_parents_age_25_above_perc,living_in_communal_establishment_uni_age_25_above_perc,living_in_communal_establishment_other_age_25_above_perc,living_in_all_student_household_age_25_above_perc,living_alone_age_25_above_perc,living_in_other_household_age_25_above_perc,total_students_2021_count,change_in_student_population_10_yr
0,E01000001,130,89,0,0,10,13,18,68.461538,0.0,...,6,9,11.764706,0.0,0.000000,0.000000,35.294118,52.941176,154,18.461538
1,E01000002,146,111,0,0,4,10,21,76.027397,0.0,...,5,14,0.000000,0.0,0.000000,5.000000,25.000000,70.000000,141,-3.424658
2,E01000003,188,78,0,65,15,16,14,41.489362,0.0,...,6,12,6.666667,0.0,13.333333,20.000000,20.000000,40.000000,137,-27.127660
3,E01000005,211,166,0,0,18,8,19,78.672986,0.0,...,3,9,26.086957,0.0,0.000000,21.739130,13.043478,39.130435,250,18.483412
4,E01000006,421,328,0,0,20,5,68,77.909739,0.0,...,3,44,10.769231,0.0,0.000000,16.923077,4.615385,67.692308,416,-1.187648
